In [1]:
#!pip install wandb

In [3]:
import pandas as pd
import numpy as np
import glob
import os
from pathlib import Path
from PIL import Image, ExifTags

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.utils import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import KFold
import scipy

import tensorflow as tf

from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from tensorflow.keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator
from tensorflow.keras.layers import Activation, Dense, GlobalAveragePooling2D, GlobalMaxPooling2D
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
import tensorflow.keras.backend as K

from tensorflow.keras.callbacks import LambdaCallback


#import wandb
#from wandb.keras import WandbCallback

#salmon-scales
from train_util import read_images, load_xy, get_checkpoint_tensorboard, create_model_grayscale, get_fresh_weights, base_output, dense1_linear_output, train_validate_test_split

print("devices:"+str(tf.test.gpu_device_name()))

devices:/device:GPU:0


In [4]:
def base_output(model):
    z = model.output
    z = GlobalMaxPooling2D()(z)
    return z

def dense1_linear_output(gray_model):
    z = base_output(gray_model)
    z = Dense(1, activation='linear')(z)
    return z

def get_checkpoint_tensorboard(tensorboard_path, checkpoint_path):
    
    tensorboard = TensorBoard(log_dir=tensorboard_path)
    checkpointer = ModelCheckpoint(
        filepath = checkpoint_path,
        verbose = 1,
        save_best_only = True,
        save_weights_only = False)
    return tensorboard, checkpointer

def train_validate_test_split(pairs, validation_set_size = 0.15, test_set_size = 0.15, a_seed = 8):
    """ split pairs into 3 set, train-, validation-, and test-set
        1 - (validation_set_size + test_set_size) = % training set size
    >>> import pandas as pd
    >>> import numpy as np
    >>> data = np.array([np.arange(10)]*2).T  # 2 columns for x, y, and one for index
    >>> df_ = pd.DataFrame(data, columns=['x', 'y'])
    >>> train_x, val_x, test_x = \
             train_validate_test_split( df_, validation_set_size = 0.2, test_set_size = 0.2, a_seed = 1 )
    >>> train_x['x'].values
    array([0, 3, 1, 7, 8, 5])
    >>> val_x['x'].values
    array([4, 6])
    >>> test_x['x'].values
    array([2, 9])
    """
    validation_and_test_set_size = validation_set_size + test_set_size
    validation_and_test_split = validation_set_size / (test_set_size+validation_set_size)
    df_train_x, df_notTrain_x = train_test_split(pairs, test_size = validation_and_test_set_size, random_state = a_seed)
    df_test_x, df_val_x = train_test_split(df_notTrain_x, test_size = validation_and_test_split, random_state = a_seed)
    return df_train_x, df_val_x, df_test_x

In [5]:
a = os.listdir( '/gpfs/gpfs0/deep/data/Savannah_Professional_Practice2021_06_10_21/CodOtholiths-MachineLearning/Savannah_Professional_Practice/2013/70021/Nr09_age02' )
b = os.listdir('/gpfs/gpfs0/deep/data/Savannah_Professional_Practice2021_06_10_21/CodOtholiths-MachineLearning/Savannah_Professional_Practice')
print(a)
print(b)

['IMG_0160.JPG', 'IMG_0161.JPG', 'IMG_0159.JPG', 'IMG_0157.JPG', 'IMG_0162.JPG', 'IMG_0158.JPG']
['2013', '2014', '2017', '2015', '2018', '2012', '2016']


In [6]:
def read_jpg_cods(B4_input_shape = (380, 380, 3), max_dataset_size = 5180, whichExposure='min'):
    #    '''
    #    reads one .jpg file in each folder in structure of folders
    #    returns tensor with images, and 1-1 correspondence with age
    #    '''

    #max_dataset_size = 5156
    #B4_input_shape = (380, 380, 3)
    df_cod = pd.DataFrame(columns=['age', 'path', 'ExposureTime'])
    image_tensor1 = np.empty(shape=(max_dataset_size,)+B4_input_shape)
    image_tensor2 = np.empty(shape=(max_dataset_size,)+B4_input_shape)
    image_tensor3 = np.empty(shape=(max_dataset_size,)+B4_input_shape)

    base_dir = '/gpfs/gpfs0/deep/data/Savannah_Professional_Practice2021_06_10_21/CodOtholiths-MachineLearning/Savannah_Professional_Practice'
    df_cod = pd.DataFrame(columns=['age', 'path', 'ExposureTime'])
    base_dirs_posix = Path(base_dir)

    error_count=0
    add_count = 0
    for some_year_dir in base_dirs_posix.iterdir():
        count = 0
        if not os.path.isdir( some_year_dir ) or "Extra" in str(some_year_dir):
            continue

        #dir structure: /year/station_number/cod_img_by_age/6 jpeg images of one fish
        stat_nos = [name for name in os.listdir( some_year_dir ) if os.path.isdir(os.path.join(some_year_dir , name))]
        for i in range(0, len(stat_nos)):
            cod_path = os.path.join( some_year_dir, stat_nos[i] )
            yr_station_codage_path = [os.path.join(cod_path , n) for n in os.listdir( cod_path ) 
                            if os.path.isdir(os.path.join(cod_path , n))]
            cod_age = [n for n in os.listdir( cod_path ) 
                            if os.path.isdir(os.path.join(cod_path , n))]

            assert len(yr_station_codage_path) == len(cod_age)
            for j in range(0, len(yr_station_codage_path)):
                #print(onlyfiles)
                onlyfiles = [f for f in os.listdir( yr_station_codage_path[j] ) 
                             if os.path.isfile(os.path.join(yr_station_codage_path[j] , f))]

                #2013/70028/Nr01_age05/Thumbs.db
                #2016/70008/Nr01_age07/Thumbs.db
                if len(onlyfiles) != 6: 
                    #print(str(len(onlyfiles)) + '\t' + str( yr_station_codage_path[j] ) + "\t" +'\t'.join(map(str,onlyfiles)))
                    error_count +=1
                else: 
                    full_path = [os.path.join(yr_station_codage_path[j] , f) 
                             for f in os.listdir( yr_station_codage_path[j] ) 
                         if os.path.isfile(os.path.join(yr_station_codage_path[j] , f))]

                    begin_age = cod_age[j].lower().find('age')
                    #print(cod_age[j])
                    age = cod_age[j][begin_age+3:begin_age+5]
                    try:
                        age = int(age)
                    except ValueError:
                        #print(yr_station_codage_path[j])
                        #print(cod_age[j])
                        #print(age)
                        #print(begin_age)
                        age = 0

                    #print(age)

                    full_path.sort()
                    exposures_set = set()
                    exposures_list = []
                    for k in range(0, len(full_path)): #len(full_path) == 6
                        img = Image.open(full_path[k])
                        exif = {ExifTags.TAGS[k]: v for k, v in img._getexif().items() if k in ExifTags.TAGS}
                        #print(exif['ExposureTime'])
                        exposures_set.add( exif['ExposureTime'] )
                        exposures_list.append( exif['ExposureTime'] )


                    #if len(exposures_set) != 3:
                        #print("\t"+str (yr_station_codage_path[j] ) + '\t' + str(exposures_list) ) 
                    #    continue
                    #else:
                    if len(exposures_list) == 6 and len(exposures_set) == 3:

                        expo_args = np.argsort(exposures_list).tolist()
                        #print( "exposures_list"+str(exposures_list) )
                        #print(" argsort: "+str(expo_args) )

                        numpy_images = [0,0,0]
                        file_paths = [0,0,0]
                        imgs_added = 0
                        
                        #use if loading to memory
                        """
                        for k in [0,2,4]:
                            img = Image.open( full_path[ expo_args[k] ] ) 
                            pil_img = load_img(full_path[ expo_args[k] ], target_size=B4_input_shape, grayscale=False)
                            array_img = img_to_array(pil_img, data_format='channels_last')

                            numpy_images[imgs_added] = array_img
                            file_paths[imgs_added] = full_path[ expo_args[k] ]
                            imgs_added += 1
                        """
                        

                        if expo_args != [1, 4, 0, 3, 2, 5]:
                            print( "exposures_list"+str(exposures_list) )
                            print(" argsort: "+str(expo_args) )
                            #print(file_paths)

                        if whichExposure == 'min':
                            #use if loading to memory
                            pil_img = load_img(full_path[ expo_args[0] ], target_size=B4_input_shape, grayscale=False)
                            array_img = img_to_array(pil_img, data_format='channels_last')
                            image_tensor1[add_count] = array_img
                            add_count += 1
                            
                            df_cod = df_cod.append({'age':age, 'path':full_path[expo_args[0]], 'light': 1, 'ExposureTime':exposures_list[expo_args[0]]}, ignore_index=True)
                        if whichExposure == 'middle':
                            #use if loading to memory
                            pil_img = load_img(full_path[ expo_args[2] ], target_size=B4_input_shape, grayscale=False)
                            array_img = img_to_array(pil_img, data_format='channels_last')
                            image_tensor1[add_count] = array_img
                            add_count += 1
                            
                            df_cod = df_cod.append({'age':age, 'path':full_path[expo_args[2]], 'light': 2, 'ExposureTime':exposures_list[expo_args[0]]}, ignore_index=True)
                        if whichExposure == 'max':
                            #use if loading to memory
                            pil_img = load_img(full_path[ expo_args[4] ], target_size=B4_input_shape, grayscale=False)
                            array_img = img_to_array(pil_img, data_format='channels_last')
                            image_tensor1[add_count] = array_img
                            add_count += 1
                            
                            df_cod = df_cod.append({'age':age, 'path':full_path[expo_args[4]], 'light': 3, 'ExposureTime':exposures_list[expo_args[0]]}, ignore_index=True)

                                        

    print("error_count:"+str(error_count))

    print("add_count:"+str(add_count))

    if whichExposure == 'min':
        return image_tensor1, df_cod
    if whichExposure == 'middle':
        return image_tensor2, df_cod
    if whichExposure == 'max':
        return image_tensor3, df_cod

    return None, None



In [7]:
tensor, df = read_jpg_cods()
#img_df = df


exposures_list[0.4, 0.2, 0.1, 0.4, 0.2, 0.1]
 argsort: [2, 5, 1, 4, 0, 3]
exposures_list[0.4, 0.2, 0.1, 0.4, 0.2, 0.1]
 argsort: [2, 5, 1, 4, 0, 3]
exposures_list[0.4, 0.2, 0.1, 0.4, 0.2, 0.1]
 argsort: [2, 5, 1, 4, 0, 3]
exposures_list[0.4, 0.2, 0.1, 0.4, 0.2, 0.1]
 argsort: [2, 5, 1, 4, 0, 3]
exposures_list[0.4, 0.2, 0.1, 0.4, 0.2, 0.1]
 argsort: [2, 5, 1, 4, 0, 3]
exposures_list[0.1, 0.4, 0.2, 0.1, 0.4, 0.2]
 argsort: [0, 3, 2, 5, 1, 4]
exposures_list[0.2, 0.1, 0.4, 0.1, 0.4, 0.2]
 argsort: [1, 3, 0, 5, 2, 4]
exposures_list[0.1, 0.4, 0.2, 0.1, 0.4, 0.2]
 argsort: [0, 3, 2, 5, 1, 4]
exposures_list[0.1, 0.4, 0.2, 0.1, 0.4, 0.2]
 argsort: [0, 3, 2, 5, 1, 4]
exposures_list[0.1, 0.4, 0.2, 0.1, 0.4, 0.2]
 argsort: [0, 3, 2, 5, 1, 4]
exposures_list[0.1, 0.4, 0.2, 0.1, 0.4, 0.2]
 argsort: [0, 3, 2, 5, 1, 4]
exposures_list[0.1, 0.4, 0.2, 0.1, 0.4, 0.2]
 argsort: [0, 3, 2, 5, 1, 4]
exposures_list[0.1, 0.4, 0.2, 0.1, 0.4, 0.2]
 argsort: [0, 3, 2, 5, 1, 4]
exposures_list[0.1, 0.4, 0.2, 0.1, 0.4

exposures_list[0.03333333333333333, 0.125, 0.06666666666666667, 0.03333333333333333, 0.125, 0.06666666666666667]
 argsort: [0, 3, 2, 5, 1, 4]
exposures_list[0.125, 0.06666666666666667, 0.03333333333333333, 0.125, 0.06666666666666667, 0.03333333333333333]
 argsort: [2, 5, 1, 4, 0, 3]
exposures_list[0.03333333333333333, 0.125, 0.06666666666666667, 0.03333333333333333, 0.125, 0.06666666666666667]
 argsort: [0, 3, 2, 5, 1, 4]
exposures_list[0.125, 0.06666666666666667, 0.03333333333333333, 0.125, 0.06666666666666667, 0.03333333333333333]
 argsort: [2, 5, 1, 4, 0, 3]
exposures_list[0.125, 0.06666666666666667, 0.03333333333333333, 0.125, 0.06666666666666667, 0.03333333333333333]
 argsort: [2, 5, 1, 4, 0, 3]
exposures_list[0.125, 0.06666666666666667, 0.03333333333333333, 0.125, 0.06666666666666667, 0.03333333333333333]
 argsort: [2, 5, 1, 4, 0, 3]
exposures_list[0.125, 0.06666666666666667, 0.03333333333333333, 0.125, 0.06666666666666667, 0.03333333333333333]
 argsort: [2, 5, 1, 4, 0, 3]
exposu

In [8]:
tensor[5113,...]

array([[[ 1.,  1.,  1.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        ...,
        [34., 29., 35.],
        [38., 32., 36.],
        [39., 34., 38.]],

       [[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        ...,
        [32., 30., 33.],
        [36., 31., 37.],
        [38., 33., 37.]],

       [[ 0.,  0.,  0.],
        [ 1.,  1.,  1.],
        [ 0.,  0.,  0.],
        ...,
        [31., 29., 32.],
        [35., 30., 34.],
        [37., 32., 36.]],

       ...,

       [[36., 31., 37.],
        [33., 30., 37.],
        [33., 30., 37.],
        ...,
        [ 1.,  1.,  1.],
        [ 1.,  1.,  1.],
        [ 0.,  2.,  1.]],

       [[33., 30., 37.],
        [34., 31., 38.],
        [32., 29., 36.],
        ...,
        [ 1.,  1.,  1.],
        [ 1.,  1.,  1.],
        [ 1.,  1.,  1.]],

       [[36., 31., 37.],
        [36., 31., 38.],
        [35., 32., 39.],
        ...,
        [ 0.,  0.,  0.],
        [ 1.,  1.,  1.],
        [ 0.,  2.,  1.]]

In [9]:
#print(tensor[5113])
tensor = tensor[0:5114]
age = df.age
image_tensor = tensor
#age
#tmp = np.asarray(age)
#unique, counts = np.unique(tmp, return_counts=True)
#print( dict(zip(unique, counts)) )
print(len(age))


5114


In [10]:
df.head()

,age,path,ExposureTime,light
0,2,/gpfs/gpfs0/deep/data/Savannah_Professional_Pr...,0.00625,1.0
1,7,/gpfs/gpfs0/deep/data/Savannah_Professional_Pr...,0.00625,1.0
2,5,/gpfs/gpfs0/deep/data/Savannah_Professional_Pr...,0.00625,1.0
3,1,/gpfs/gpfs0/deep/data/Savannah_Professional_Pr...,0.00625,1.0
4,8,/gpfs/gpfs0/deep/data/Savannah_Professional_Pr...,0.00625,1.0


In [11]:
loadImgToMem=True
os.environ["CUDA_VISIBLE_DEVICES"]="0"
tensorboard_path= './tensorboard_test3'
checkpoint_path = './checkpoints_test3/cod_oto_efficientnetBBB.{epoch:03d}-{val_loss:.2f}.hdf5'
a_batch_size = 16
B4_input_shape = (380, 380, 3)
B5_input_shape = (456, 456, 3)

new_shape = B4_input_shape

#new_shape = B5_input_shape
#B4_input_shape = B5_input_shape

#image_tensor, age = read_jpg_cods(B4_input_shape)

In [ ]:
#image_tensor[5178,:,:,0]

In [12]:
image_tensor.shape

(5114, 380, 380, 3)

### Model config

In [13]:
#train_datagen = ImageDataGenerator(
#    zca_whitening=False,
#    width_shift_range=0.2,
#    height_shift_range=0.2, #20,
#    #zoom_range=[0.5,1.0],
#    rotation_range=360,
#    horizontal_flip=False,
#    vertical_flip=True,
#    rescale=1./255)

train_datagen = ImageDataGenerator(
    zca_whitening=False,
    width_shift_range=0.0,
    height_shift_range=0.0,
    rotation_range=0,
    horizontal_flip=False,
    vertical_flip=False) #,
    #rescale=1./255)

In [14]:
#Tensorflow 2.2 - wrap generator in tf.data.Dataset   
def callGen():
    return train_datagen.flow(train_rb_imgs, train_age, batch_size=a_batch_size)

In [20]:
learning_rate=0.00005
learning_rate=0.006
adam = optimizers.Adam(learning_rate=learning_rate)

early_stopper = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, verbose=0,
                                      mode='min',restore_best_weights=True)

plateau = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, 
                                               verbose=0,mode='min')
        
def binary_accuracy_for_regression(y_true, y_pred):
    return K.mean(K.equal(y_true, K.round(y_pred)), axis=-1)

classWeight = None

######## Define Model ######
"""
img_augmentation = Sequential([
        layers.RandomRotation(factor=0.15),
        layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
        layers.RandomFlip(),
        layers.RandomContrast(factor=0.1),
    ],
    name="img_augmentation",
)
"""

rescaleLayer = tf.keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255)],
  name="rescaling1")
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal"),
  layers.experimental.preprocessing.RandomRotation(factor=(-0.5, 0.5))],
  name="img_augmentation",
  # possible width shift, height shift  
)

rgb_efficientNetB4 = tf.keras.applications.EfficientNetB4(
    include_top=False, weights='imagenet', input_shape=B4_input_shape, classes=2)

inputLayer = layers.Input( shape = B4_input_shape )
x = rescaleLayer(inputLayer)
x = data_augmentation(x)
x = rgb_efficientNetB4(x)
#z = rgb_efficientNetB4.output
z = GlobalMaxPooling2D()(x)
z = Dense(1, activation='linear')(z)

cod = Model( inputs = inputLayer, outputs = z)
print( cod.summary() )

#z = rgb_efficientNetB4.output
#z = GlobalMaxPooling2D()(z)
#z = Dense(1, activation='linear')(z)
#cod = Model( inputs = inputLayer, outputs = z)
#print(cod.summary())

cod.compile(loss='mse', optimizer=adam, metrics=['mse', binary_accuracy_for_regression] )
tensorboard, checkpointer = get_checkpoint_tensorboard(tensorboard_path, checkpoint_path)
K.set_value(cod.optimizer.learning_rate, 0.005)

for layer in cod.layers:
    layer.trainable = True
############################

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 380, 380, 3)]     0         
_________________________________________________________________
rescaling1 (Sequential)      (None, 380, 380, 3)       0         
_________________________________________________________________
img_augmentation (Sequential (None, 380, 380, 3)       0         
_________________________________________________________________
efficientnetb4 (Functional)  (None, 12, 12, 1792)      17673823  
_________________________________________________________________
global_max_pooling2d_1 (Glob (None, 1792)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1793      
Total params: 17,675,616
Trainable params: 17,550,409
Non-trainable params: 125,207
_________________________________________

### KFold out-of-Fold split

In [16]:
train_idx, val_idx, test_idx = train_validate_test_split( range(0, len(image_tensor)) )
train_idx_oof = train_idx + val_idx #prepare - train+validation set for 5 KFold cv split
val_idx = None

#if train in memory
train_rb_imgs = np.empty(shape=(len(train_idx),)+B4_input_shape)
test_rb_imgs = np.empty(shape=(len(test_idx),)+B4_input_shape)
train_age = []

for i in range(0, len(train_idx)):
    train_rb_imgs[i] = image_tensor[train_idx[i]]
    train_age.append(age[train_idx[i]])

test_age = []
for i in range(0, len(test_idx)):
    test_rb_imgs[i] = image_tensor[test_idx[i]]
    test_age.append(age[test_idx[i]])    
    
test_rb_imgs = np.multiply(test_rb_imgs, 1./255)    
print(train_rb_imgs.shape)

(3579, 380, 380, 3)


In [17]:
# K-fold
a_seed=2021
kfold = KFold(n_splits = 5, random_state = a_seed, shuffle = True)

#train_oof = image_tensor[train_idx_oof,:,:]
#train_oof.shape

#train_idx_oof = tr
#train_img_tensor = 
#for fold, (trn_ind, val_ind) in enumerate(kfold.split(train)):

In [ ]:
# 1. Start a new run
#wandb.init(project="cod_oto_B4")
# 2. Save model inputs and hyperparameters
#config = wandb.config
#config.learning_rate = 0.00001

the_fold = 0
for fold, (trn_ind, val_ind) in enumerate(kfold.split(train_age)):
    train_idx = trn_ind
    val_idx = val_ind
    
    X_train, X_val = image_tensor[trn_ind,:,:,:], image_tensor[val_ind,:,:,:]
    print(X_train.shape)
    print(X_val.shape)
    print("range train_age"+str(max(trn_ind)))
    print("len train_age:"+str(len(train_age)))
    print("len trn_ind:"+str(len(trn_ind)))
    print("len val_ind:"+str(len(val_ind)))
    print("len trn_ind + val_ind:"+str(len(trn_ind)+len(val_ind)))
    print(type(trn_ind))
    new_train_age = np.array(train_age)
    y_train = new_train_age[trn_ind]
    print(y_train)
    y_val = new_train_age[val_ind]
    #y_train = y_train.tolist()
    #y_val = y_val.tolist()
    
    print(new_train_age.shape)
    print(X_train.shape)
    print(type(y_train))
    print(y_train)
    print(y_train.shape)
    
    #use when imgs in memory
    #Tensorflow 2.2 - wrap generator in tf.data.Dataset
    def callGen():
        return train_datagen.flow(X_train, y_train, batch_size=a_batch_size) #train_age

    train_dataset = tf.data.Dataset.from_generator(callGen, 
                                   (tf.float32, tf.float32)).shuffle(
                                        128, reshuffle_each_iteration=True).repeat()


    log_file_name = 'loss_log/loss_log_'+str(the_fold)+'.txt'
    print( "log_file_name:"+str(log_file_name) )
    txt_log = open(log_file_name, mode='wt', buffering=1)

    save_op_callback = LambdaCallback(
      on_epoch_end = lambda epoch, logs: txt_log.write(
        str( {'epoch': epoch, 'loss': logs['loss']} ) + '\n'),
      on_train_end = lambda logs: txt_log.close()
    )

    print("training the fold:"+str(the_fold))
    the_fold += 1
    history_callback = cod.fit(train_dataset ,
        steps_per_epoch=1600,
        epochs=1000,
        callbacks=[early_stopper, plateau, save_op_callback], #, tensorboard, checkpointer],WandbCallback()
        #validation_data = val_dataset,
        validation_data = (X_val, y_val), #val_age
        class_weight=classWeight,
        workers=4,
        verbose=1,
        use_multiprocessing=True)

    #test_metrics = cod.evaluate_generator(test_rb_imgs)
    test_metrics = cod.evaluate(x=test_rb_imgs, y=test_age)
    print("test metric:"+str(cod.metrics_names))
    print("test metrics:"+str(test_metrics))

    print("precision, recall, f1")
    y_pred_test = cod.predict(test_rb_imgs, verbose=1)
    y_pred_test_bool = np.argmax(y_pred_test, axis=1)
    y_true_bool = np.argmax(test_age, axis=1)
    #np.argmax inverse of to_categorical
    argmax_test = np.argmax(test_age, axis=1)
    unique, counts = np.unique(argmax_test, return_counts=True)
    print("test ocurrence of each class:"+str(dict(zip(unique, counts))))

    print("cslassification_report")
    print(classification_report(y_true_bool, y_pred_test_bool))
    print("confusion matrix")
    print(str(confusion_matrix(y_true_bool, y_pred_test_bool)))


(2863, 380, 380, 3)
(716, 380, 380, 3)
range train_age3577
len train_age:3579
len trn_ind:2863
len val_ind:716
len trn_ind + val_ind:3579
<class 'numpy.ndarray'>
[6 6 1 ... 3 2 6]
(3579,)
(2863, 380, 380, 3)
<class 'numpy.ndarray'>
[6 6 1 ... 3 2 6]
(2863,)
log_file_name:loss_log/loss_log_0.txt
training the fold:0
Epoch 1/1000
1600/1600 [==============================] - 809s 496ms/step - loss: 13.2250 - mse: 13.2240 - binary_accuracy_for_regression: 0.1253 - val_loss: 8.3203 - val_mse: 8.3203 - val_binary_accuracy_for_regression: 0.1187
Epoch 2/1000
1600/1600 [==============================] - 787s 492ms/step - loss: 8.1383 - mse: 8.1379 - binary_accuracy_for_regression: 0.1299 - val_loss: 7.6280 - val_mse: 7.6280 - val_binary_accuracy_for_regression: 0.1648
Epoch 3/1000
1600/1600 [==============================] - 788s 492ms/step - loss: 8.1243 - mse: 8.1244 - binary_accuracy_for_regression: 0.1262 - val_loss: 10.0930 - val_mse: 10.0930 - val_binary_accuracy_for_regression: 0.1117
Ep

In [102]:
unique, counts = np.unique(age, return_counts=True)
print(dict(zip(unique, counts)))
unique, counts = np.unique(train_age, return_counts=True)
print(dict(zip(unique, counts)))

{0: 11, 1: 380, 2: 509, 3: 504, 4: 617, 5: 795, 6: 532, 7: 544, 8: 476, 9: 319, 10: 214, 11: 120, 12: 54, 13: 26, 14: 7, 15: 4, 16: 1, 17: 1}
{0: 6, 1: 246, 2: 351, 3: 336, 4: 442, 5: 542, 6: 395, 7: 372, 8: 341, 9: 226, 10: 164, 11: 86, 12: 44, 13: 19, 14: 5, 15: 3, 16: 1}


In [ ]:
    ############## salmon_scales
    ''
    new_shape = (380, 380, 3)
    age_cod = age
    rb_imgs, all_sea_age, all_smolt_age, all_farmed_class, all_spawn_class, all_filenames = load_xy()
    uten_ukjent = len(all_sea_age) - all_sea_age.count(-1.0)
    rb_imgs2 = np.empty(shape=(uten_ukjent,)+new_shape)
    unique, counts = np.unique(all_sea_age, return_counts=True)
    print("age distrib:"+str( dict(zip(unique, counts)) ))

    all_sea_age2 = []
    found_count = 0
    all_filenames2 = []
    for i in range(0, len(all_sea_age)):
        if all_sea_age[i] > -1:
            rb_imgs2[found_count] = rb_imgs[i]
            all_sea_age2.append(all_sea_age[i])
            found_count += 1
            all_filenames2.append(all_filenames[i])

    assert found_count == uten_ukjent

    age_scales = all_sea_age2
    rb_imgs = rb_imgs2

    age_scales = np.vstack(age_scales)

    train_datagen_scales = ImageDataGenerator(
        zca_whitening=False,
        width_shift_range=0.2,
        height_shift_range=0.2, #20,
        #zoom_range=[0.5,1.0],
        rotation_range=360,
        horizontal_flip=False,
        vertical_flip=True,
        rescale=1./255)

    train_generator_scales = train_datagen_scales.flow(rb_imgs, age_scales, batch_size= a_batch_size)
    history_callback_scales = cod.fit(train_generator_scales,
        steps_per_epoch=1000,
        epochs=20,
        #callbacks=[early_stopper, tensorboard, checkpointer],
        #validation_data= (val_rb_imgs, val_age),
        class_weight=classWeight)
    ######################



In [ ]:

    K.set_value(cod.optimizer.learning_rate, 0.00001)
    print("Learning rate before second fit:", cod.optimizer.learning_rate.numpy())

    history_callback = cod.fit(train_dataset ,
        steps_per_epoch=1600,
        epochs=150,
        callbacks=[early_stopper, tensorboard, checkpointer],
        #validation_data = val_dataset,
        validation_data = (val_rb_imgs, val_age),
        class_weight=classWeight)

    #test_metrics = cod.evaluate_generator(test_dataset)
    test_metrics = cod.evaluate(x=test_rb_imgs, y=test_age)
    print("test metric:"+str(cod.metrics_names))
    print("test metrics:"+str(test_metrics))

    print("precision, recall, f1")
    y_pred_test = cod.predict(test_rb_imgs, verbose=1)
    y_pred_test_bool = np.argmax(y_pred_test, axis=1)
    y_true_bool = np.argmax(test_age, axis=1)
    #np.argmax inverse of to_categorical
    argmax_test = np.argmax(test_age, axis=1)
    unique, counts = np.unique(argmax_test, return_counts=True)
    print("test ocurrence of each class:"+str(dict(zip(unique, counts))))

    print("cslassification_report")
    print(classification_report(y_true_bool, y_pred_test_bool))
    print("confusion matrix")
    print(str(confusion_matrix(y_true_bool, y_pred_test_bool)))




